Inventory of SQL permisions by User 

# SQL Inventory

In [ ]:
import pyodbc
import yaml
driver= '{ODBC Driver 17 for SQL Server}'
try:
    config = yaml.safe_load(open("config.yml"))
except yaml.YAMLError as error:
    print(error)
username = config["username"]
password = config["password"]

In [ ]:
sqlservers = [
    'server1',
    'server2',
    'server3'
    ]
sqlservers

## List all roles

In [ ]:
for sqlserver in sqlservers:
    server = sqlserver + '.database.windows.net'

    cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE=master;UID='+username+';PWD='+ password+';Authentication={ActiveDirectoryPassword}')

    cursor = cnxn.cursor()
    cursor.execute("SELECT name FROM master.sys.databases")
    dbs = cursor.fetchall()
    for db in dbs:
        cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+str(db[0])+';UID='+username+';PWD='+ password+';Authentication={ActiveDirectoryPassword}')
        cursor = cnxn.cursor()

        cursor.execute("select A.name, A.type_desc, A.principal_id, B.member_principal_id, C.uid, C.name FROM sys.database_principals AS A LEFT JOIN sys.database_role_members AS B ON A.principal_id = B.member_principal_id LEFT JOIN sysusers AS C ON B.role_principal_id = C.uid ORDER BY type_desc,A.name")
        rows = cursor.fetchall()
        for row in rows:
            print(sqlserver + "," + str(db[0]) + "," + str(row[0]) + "," + str(row[1]) + "," + str(row[5]))

        cnxn.close()

## List all granular permisions

In [ ]:
for sqlserver in sqlservers:
    server = sqlserver + '.database.windows.net'

    cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE=master;UID='+username+';PWD='+ password+';Authentication={ActiveDirectoryPassword}')

    cursor = cnxn.cursor()
    cursor.execute("SELECT name FROM master.sys.databases")
    dbs = cursor.fetchall()
    for db in dbs:
        cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+str(db[0])+';UID='+username+';PWD='+ password+';Authentication={ActiveDirectoryPassword}')
        cursor = cnxn.cursor()

        cursor.execute("SELECT DB_NAME() AS DatabaseName, DatabasePrincipals.name AS PrincipalName, DatabasePrincipals.type_desc AS PrincipalType, DatabasePrincipals2.name AS GrantedBy, DatabasePermissions.permission_name AS Permission, DatabasePermissions.state_desc AS StateDescription, SCHEMA_NAME(SO.schema_id) AS SchemaName,SO.Name AS ObjectName, SO.type_desc AS ObjectType FROM sys.database_permissions DatabasePermissions LEFT JOIN sys.objects SO ON DatabasePermissions.major_id = so.object_id LEFT JOIN sys.database_principals DatabasePrincipals ON DatabasePermissions.grantee_principal_id = DatabasePrincipals.principal_id LEFT JOIN sys.database_principals DatabasePrincipals2 ON DatabasePermissions.grantor_principal_id = DatabasePrincipals2.principal_id")
        rows = cursor.fetchall()
        for row in rows:
            print(sqlserver + "," + str(db[0]) + "," + str(row[0]) + "," + str(row[1]) + "," + str(row[2]) + "," + str(row[3]) + "," + str(row[4]) + "," + str(row[5]) + "," + str(row[6]) + "," + str(row[7]) + "," + str(row[8]))
        cnxn.close()

# ASSIGN PERMISIONS

In [ ]:
client = 'IT-GLB-EYMP_OPS_L3_MSP01-Team'

for sqlserver in sqlservers:
    server = sqlserver + '.database.windows.net'

    cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE=master;UID='+username+';PWD='+ password+';Authentication={ActiveDirectoryPassword};Connection Timeout=360;autocommit=True')

    cursor = cnxn.cursor()
    cursor.execute("SELECT name FROM master.sys.databases")
    dbs = cursor.fetchall()
    
    try:
        cursor.execute("CREATE USER [" + client + "] FROM EXTERNAL PROVIDER WITH DEFAULT_SCHEMA = [dbo]")
        cursor.commit()
    except pyodbc.Error as ex:
        if ex.args[0] != '42000':
            print(ex.args)

    for db in dbs:
        if str(db[0]) != 'master':
            cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+str(db[0])+';UID='+username+';PWD='+ password+';Authentication={ActiveDirectoryPassword};Connection Timeout=360')
            cursor = cnxn.cursor()

            try:
                cursor.execute("CREATE USER [" + client + "] FROM EXTERNAL PROVIDER")
            except pyodbc.Error as ex:
                if ex.args[0] != '42000':
                    print(ex.args)
        
            cursor.execute('GRANT EXEC TO [' + client + ']')
            cursor.execute("EXEC sp_addrolemember N'db_datareader', [" + client + "]; EXEC sp_addrolemember N'db_datawriter', [" + client + "]; EXEC sp_addrolemember N'db_ddladmin', [" + client + "];")
            cursor.commit()

        cursor.execute("select A.name, A.type_desc, A.principal_id, B.member_principal_id, C.uid, C.name FROM sys.database_principals AS A LEFT JOIN sys.database_role_members AS B ON A.principal_id = B.member_principal_id LEFT JOIN sysusers AS C ON B.role_principal_id = C.uid WHERE A.name = '" + client + "' ORDER BY type_desc,A.name")
        rows = cursor.fetchall()
        for row in rows:
            print("Role," + sqlserver + "," + str(db[0]) + "," + str(row[0]) + "," + str(row[1]) + "," + str(row[5]))

        cursor.execute("SELECT DB_NAME() AS DatabaseName, DatabasePrincipals.name AS PrincipalName, DatabasePrincipals.type_desc AS PrincipalType, DatabasePrincipals2.name AS GrantedBy, DatabasePermissions.permission_name AS Permission, DatabasePermissions.state_desc AS StateDescription, SCHEMA_NAME(SO.schema_id) AS SchemaName,SO.Name AS ObjectName, SO.type_desc AS ObjectType FROM sys.database_permissions DatabasePermissions LEFT JOIN sys.objects SO ON DatabasePermissions.major_id = so.object_id LEFT JOIN sys.database_principals DatabasePrincipals ON DatabasePermissions.grantee_principal_id = DatabasePrincipals.principal_id LEFT JOIN sys.database_principals DatabasePrincipals2 ON DatabasePermissions.grantor_principal_id = DatabasePrincipals2.principal_id Where DatabasePrincipals.name='" + client + "'")
        rows = cursor.fetchall()
        for row in rows:
            print("Permission," + sqlserver + "," + str(db[0]) + "," + str(row[0]) + "," + str(row[1]) + "," + str(row[2]) + "," + str(row[3]) + "," + str(row[4]) + "," + str(row[5]) + "," + str(row[6]) + "," + str(row[7]) + "," + str(row[8]))
        cnxn.close()